In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ROOT_PATH = r"/content/drive/MyDrive/Document/EdgeAI/Data/FallDetection"

Mounted at /content/drive


In [ ]:
from builtins import print
import numpy as np
import os
import plotly.graph_objects as go
from keras.models import load_model
import IPython
import glob
import random
import math
import pandas as pd

In [ ]:
model_path = os.path.join(ROOT_PATH, r'models/model_Colab1.h5')
model = load_model(model_path)
labels = ["Norm", "Fall"]

In [ ]:
def feature_math(slide_df):
  x_std = slide_df.std()['x']
  y_std = slide_df.std()['y']
  z_std = slide_df.std()['z']

  x_mean = slide_df.mean()['x']
  y_mean = slide_df.mean()['y']
  z_mean = slide_df.mean()['z']
  
  x_skew = slide_df.skew()['x']
  y_skew = slide_df.skew()['y']
  z_skew = slide_df.skew()['z']
  
  return [x_std, y_std, z_std, x_mean, y_mean, z_mean, x_skew, y_skew, z_skew]

In [ ]:
def softmax(x):
    return(np.exp(x)/np.exp(x).sum())

In [ ]:
file_path_lst = glob.glob(os.path.join(ROOT_PATH, r'Data/raw/fall*.csv'))
random.shuffle(file_path_lst)
file_path = file_path_lst[0]

print(file_path)

frame_length = 15
overlap_ratio = 0.3
hop_length = int(frame_length * (1 - overlap_ratio))


df = pd.read_csv(file_path, names=['time', 'total', 'x', 'z', 'y'])
clip_length = df.count()['x']
slide_nums = math.floor(clip_length / hop_length) - 1
y_lst = []
for i in range(slide_nums):  
  idx_min, idx_max = (i*hop_length), (i*hop_length + frame_length)
  slide_df = df[(df.index >= idx_min) & (df.index < idx_max)]
  sensor_in = np.expand_dims(feature_math(slide_df), axis=0)
  y_probs = model.predict(sensor_in, verbose=0)
  # slide_df.insert(5, 'fall/norm', np.ones(slide_df.count()['x']) * np.argmax(softmax(y_probs), axis=1).tolist())
  print(y_probs)
  y_probs = (np.ones(slide_df.count()['x']) * np.argmax(softmax(y_probs), axis=1)).tolist()
  # print(slide_df.count()['x'])
  if i > 0:
    y_lst = y_lst[:idx_min]
    y_lst += y_probs
  else:
    y_lst = y_probs
  # df_lst.append(slide_df)
# df_lst = pd.concat(df_lst)

df = df[df.index < len(y_lst)]
df.insert(5, 'fall/norm', y_lst)

/content/drive/MyDrive/Document/EdgeAI/Data/FallDetection/Data/raw/fall-13-acc.csv
[[0.91247886 0.35230535]]
[[0.74777424 0.40993565]]
[[0.96116245 0.35547125]]
[[0.768592   0.38919964]]
[[0.779416   0.39037117]]
[[0.7760779  0.37041855]]
[[0.71418494 0.43110898]]
[[0.73044276 0.41153756]]
[[0.70258284 0.44110405]]
[[0.68452793 0.4236604 ]]
[[0.7047065  0.43830016]]
[[0.7506903  0.39826056]]
[[0.7440972 0.4092358]]
[[0.7721175  0.43120164]]
[[0.2627512 0.935749 ]]
[[0.44203746 0.82823914]]
[[0.02001552 0.9975221 ]]
[[0.00816508 0.99929404]]
[[0.14447848 0.95355624]]
[[0.496425  0.7607585]]
[[0.6158256 0.576439 ]]
[[0.65984946 0.5264077 ]]
[[0.71915567 0.4482871 ]]
[[0.7236006 0.4306694]]


In [ ]:
fig = go.Figure([
    go.Scatter(x = df['time'], y = df['total'], name='Total'),
    go.Scatter(x = df['time'], y = df['x'], name='X'),
    go.Scatter(x = df['time'], y = df['y'], name='Y'),
    go.Scatter(x = df['time'], y = df['z'], name='Z'),
    go.Scatter(x = df['time'], y = df['fall/norm'], name='fall/norm'),
    ])

fig.update_layout(title='Normal' if 'adl' in file_path else 'Fall',
                   plot_bgcolor='rgb(230, 230,230)',
                   showlegend=True)

fig.show()